## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-14-50-48 +0000,nyt,"Live Updates: Backed By Allies, Zelensky Heads...",https://www.nytimes.com/live/2025/08/18/us/tru...
1,2025-08-18-14-49-26 +0000,nyt,Seven European Leaders Rush to Join Zelensky a...,https://www.nytimes.com/2025/08/18/us/european...
2,2025-08-18-14-47-59 +0000,nyt,"Oil and Gas Prices Are Falling, a Boon for Dri...",https://www.nytimes.com/2025/08/18/business/en...
3,2025-08-18-14-44-25 +0000,bbc,US East Coast faces life-threatening waves fro...,https://www.bbc.com/news/articles/c3r41zeqnexo...
4,2025-08-18-14-44-00 +0000,wsj,Texas Democrats ended a two-week walkout over ...,https://www.wsj.com/politics/policy/texas-demo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,40
4,zelensky,20
185,ukraine,18
223,putin,14
78,canada,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,130
64,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,116
53,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,110
37,2025-08-18-13-00-25 +0000,nyt,Zelensky Brings Backup to the White House as T...,https://www.nytimes.com/2025/08/17/us/politics...,99
66,2025-08-18-10-10-29 +0000,nyt,Zelensky Calls for ‘Lasting’ Peace in Ukraine ...,https://www.nytimes.com/2025/08/18/us/politics...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,130,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
201,51,2025-08-17-16-19-12 +0000,nypost,Rubio declares Russia has to ‘get something’ f...,https://nypost.com/2025/08/17/us-news/rubio-de...
17,42,2025-08-18-14-15-38 +0000,nypost,Illegal migrant suspect in deadly Florida cras...,https://nypost.com/2025/08/18/us-news/illegal-...
22,40,2025-08-18-13-53-28 +0000,cbc,Federal labour board deems Air Canada flight a...,https://www.cbc.ca/news/politics/air-canada-st...
45,32,2025-08-18-12-31-57 +0000,nypost,Texas ICE shooting suspects tied to secretive ...,https://nypost.com/2025/08/18/us-news/texas-ic...
155,30,2025-08-17-21-45-24 +0000,nypost,New Yorkers fighting against massive battery s...,https://nypost.com/2025/08/17/us-news/new-york...
193,29,2025-08-17-17-18-00 +0000,wsj,More National Guard troops are heading soon to...,https://www.wsj.com/politics/national-security...
15,25,2025-08-18-14-20-22 +0000,nypost,Hurricane Erin’s life-threatening impacts prom...,https://nypost.com/2025/08/18/us-news/hurrican...
195,24,2025-08-17-17-01-21 +0000,cbc,European leaders to join Zelenskyy's meeting w...,https://www.cbc.ca/news/world/european-leaders...
18,23,2025-08-18-14-09-13 +0000,nypost,Moped-riding perp shot dead after pulling gun ...,https://nypost.com/2025/08/18/us-news/moped-ri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
